# Convert Wiki docs to Dict - JSON file

In [1]:
import time
import glob
import string
import sys
import pickle
import pandas as pd
import io

In [2]:
path = "wiki-pages-text"
files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]

In [3]:
wikiData = {}
for i, shard_path in enumerate(files):
    print("FILE PROCESSED: ",i+1,"\t",shard_path)
    fp = open(shard_path,'r',encoding="utf-8")
    text = fp.readlines()
    for line_num, line in enumerate(text):
        line_list = line.split(" ")
        page_id = line_list.pop(0)
        try:
            sentence_id = int(line_list[0])
        except:
            pass
        sentence_text = " ".join(line_list[1:])
        sentence_text = sentence_text.replace("\n","")
        
        wikiData[(page_id,sentence_id)] = sentence_text

FILE PROCESSED:  1 	 wiki-pages-text/wiki-009.txt
FILE PROCESSED:  2 	 wiki-pages-text/wiki-021.txt
FILE PROCESSED:  3 	 wiki-pages-text/wiki-035.txt
FILE PROCESSED:  4 	 wiki-pages-text/wiki-034.txt
FILE PROCESSED:  5 	 wiki-pages-text/wiki-020.txt
FILE PROCESSED:  6 	 wiki-pages-text/wiki-008.txt
FILE PROCESSED:  7 	 wiki-pages-text/wiki-036.txt
FILE PROCESSED:  8 	 wiki-pages-text/wiki-022.txt
FILE PROCESSED:  9 	 wiki-pages-text/wiki-023.txt
FILE PROCESSED:  10 	 wiki-pages-text/wiki-037.txt
FILE PROCESSED:  11 	 wiki-pages-text/wiki-033.txt
FILE PROCESSED:  12 	 wiki-pages-text/wiki-027.txt
FILE PROCESSED:  13 	 wiki-pages-text/wiki-026.txt
FILE PROCESSED:  14 	 wiki-pages-text/wiki-032.txt
FILE PROCESSED:  15 	 wiki-pages-text/wiki-024.txt
FILE PROCESSED:  16 	 wiki-pages-text/wiki-030.txt
FILE PROCESSED:  17 	 wiki-pages-text/wiki-018.txt
FILE PROCESSED:  18 	 wiki-pages-text/wiki-019.txt
FILE PROCESSED:  19 	 wiki-pages-text/wiki-031.txt
FILE PROCESSED:  20 	 wiki-pages-text/wi

In [4]:
wikiData = {str(key):val for key,val in wikiData.items()}

## Save dictionary with name as data to a pickle file

In [ ]:
with open('dictPickle.pickle', 'wb') as handle:  
    pickle.dump(wikiData, handle)

## Read the pickle file into a dictionary with name of wikiDict:

In [36]:
with open('dictPickle.pickle', 'rb') as handle:
    wikiTrain = pickle.load(handle)

In [40]:
import unicodedata
def normalize(inputString):
    intermediaryString = unicodedata.normalize('NFKD', inputString)
    return intermediaryString

In [41]:
train_file = "train.json"
trainset = pd.read_json(train_file,orient='index')

In [45]:
# Normalise the key string b4 adding them as keys
wikiTrain = {normalize(key):val for key,val in wikiTrain.items()}
textList = []
missed_keys= []
for data in trainset['evidence']:
    textData = []
    for val in data:
        pair = str(tuple(val))
        # normalise the key pair and query it
        textData.append(normalize(wikiTrain[normalize(pair)]))
    textList.append(textData)
    
trainset['text'] = textList

## Train Set with Text

In [51]:
trainset.to_json('train_with_text.json')